**PDF TO IMAGE TO PDF CONVERTER**

Install modules

In [ ]:
!pip install pdf2image
!pip install wand
!pip install fpdf
!pip install img2pdf
!pip install PyPDF2
!apt-get install PIL
!apt-get install poppler-utils
!apt-get install libmagickwand-dev
from google.colab import drive
drive.mount('/content/drive')

Create paths + Mount google drive

In [ ]:
import os

# Mount Google drive
path = '/content/drive/Targetfolder/'
imgpath = path + 'image/'
pdfpath = path + 'pdf/'
gradedpath = path + 'graded/'

if not os.path.exists(imgpath):
    os.makedirs(imgpath)
if not os.path.exists(pdfpath):
    os.makedirs(pdfpath)
if not os.path.exists(gradedpath):
    os.makedirs(gradedpath)

files = os.listdir(path)
for i in files:
  print(i)

PDF to IMAGE converter

In [ ]:
from pdf2image import convert_from_path
from PIL import Image

for filename in os.listdir(path):
    basename, extension = os.path.splitext(filename)
    outfilename = imgpath+filename.split("_")[0]
    print(basename, extension)
    if extension == '.pdf': 
      # print(extension, basename)
      dpi = 500 # dots per inch
      pdf_file = path + filename
      pages = convert_from_path(pdf_file ,dpi )
      for i in range(len(pages)):
        page = pages[i]
        page.save(outfilename+'_{}.jpeg'.format(i), 'JPEG')
    elif extension in ['.jpg','.png','.jpeg'] :
      image = Image.open(path + filename)
#             image.alpha_channel = 'remove'
      image.save(imgpath + basename + '.jpeg')
      image.close()
    else:
      continue

Create PDF file from each image

In [ ]:
import fpdf 
import img2pdf           
            
for filename in os.listdir(imgpath):
    basename, extension = os.path.splitext(filename)
    pdf_path = pdfpath + basename + '.pdf' # new pdf path       
    if not os.path.exists(pdf_path):
        image = Image.open(imgpath + filename) # opening image 
        # print(filename)
        pdf_bytes = img2pdf.convert(image.filename) # converting into chunks using img2pdf 
        file = open(pdf_path, "wb") # opening or creating pdf file
        file.write(pdf_bytes) # writing pdf files with chunks
        image.close() # closing image file 
        file.close() # closing pdf file

Merge PDF file using student ID

In [ ]:
from os import listdir
from os.path import isfile, join
from collections import defaultdict
import difflib

# Group files with same student ID (first 9 characters)
onlyfiles = [f for f in listdir(pdfpath)]
# print("\n".join(onlyfiles))

groups = defaultdict(list)
for filename in os.listdir(pdfpath):
    basename, extension = os.path.splitext(filename)
    if extension == '.pdf':
        groups[filename[:9]].append(filename)
        
# Merge pdf files or copy one file to new folder  
from fpdf import FPDF     
import PyPDF2
import shutil

for key, filename in groups.items():  
    # print(key,filename)
    
    outfilename = gradedpath+filename[0].split("_")[0]+_Graded.pdf'
    if not os.path.exists(outfilename):
        # print(len(filename))
        if len(filename)>1:
            pdfWriter = PyPDF2.PdfFileWriter()
            for file in filename:                            
                # print(file)       
                pdfFileObj = open(pdfpath+file,'rb') #rb for read binary
                pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
                #Opening each page of the PDF
                for pageNum in range(pdfReader.numPages):
                    pageObj = pdfReader.getPage(pageNum)
                    pdfWriter.addPage(pageObj)        
            pdfOutput = open(outfilename, 'wb') #save PDF to file, wb for write binary
            pdfWriter.write(pdfOutput) #Outputting the PDF        
            pdfOutput.close() #Closing the PDF writer
        else:
            shutil.copy(pdfpath+filename[0], outfilename)    